### Combine Players FPL Price for Specific Gameweek

In [1]:
import os
import pandas as pd

def combine_dataset(folder, position):
    # Tentukan path folder yang berisi file Excel
    folder_path = folder

    # List untuk menyimpan data dari setiap file
    all_data = []

    # Loop melalui setiap file dalam folder
    for file in os.listdir(folder_path):
        # Cek apakah file adalah file Excel
        if file.endswith('.xlsx'):
            # Baca file Excel
            file_path = os.path.join(folder_path, file)
            df = pd.read_excel(file_path, header=None)  # Membaca tanpa header
            
            # Tentukan nama header baru
            df.columns = ['name', 'team', 'price', 'condition']
            
            # Fungsi untuk mengubah nilai pada kolom 'condition'
            def update_condition(value):
                if '25% chance of playing' in value:
                    return 0.25
                elif '50% chance of playing' in value:
                    return 0.5
                elif '75% chance of playing' in value:
                    return 0.75
                elif 'Unlikely to play' in value:
                    return 0
                else:
                    return 1
                
            # Terapkan fungsi ke kolom 'condition'
            df['condition'] = df['condition'].apply(update_condition)
            
            # Menambahkan data ke dalam list
            all_data.append(df)

    # Menggabungkan semua dataframe
    df = pd.concat(all_data, ignore_index=True)
    df['position'] = position
    
    return df

In [2]:
gw_no = 3
gw = f'gw-{gw_no}'

gk_df = combine_dataset(f'{gw}/gk', 'GK')
def_df = combine_dataset(f'{gw}/def', 'DEF')
mid_df = combine_dataset(f'{gw}/mid', 'MID')
fwd_df = combine_dataset(f'{gw}/fwd', 'FWD')

df = pd.concat([gk_df, def_df, mid_df, fwd_df]).reset_index(drop=True)

In [3]:
team_map = {
    'ARS':'Arsenal', 
    'AVL':'Aston Villa', 
    'BHA':'Brighton & Hove Albion', 
    'BOU':'Bournemouth', 
    'BRE':'Brentford', 
    'CHE':'Chelsea', 
    'CRY':'Crystal Palace', 
    'EVE':'Everton', 
    'FUL':'Fulham',
    'IPS':'Ipswich Town', 
    'LEI':'Leicester City', 
    'LIV':'Liverpool', 
    'MCI':'Manchester City', 
    'MUN':'Manchester United', 
    'NEW':'Newcastle United', 
    'NFO':'Nottingham Forest', 
    'SOU':'Southampton', 
    'TOT':'Tottenham Hotspur',
    'WHU':'West Ham United', 
    'WOL':'Wolverhampton Wanderers',
}

df['team'] = df['team'].replace(team_map)
df['GW'] = gw_no

In [4]:
df.to_csv(f'price/fpl-price-{gw}.csv', index=False)

### Merge FPL Price per Gameweek

In [5]:
import pandas as pd
import os

# Tentukan path folder yang berisi file CSV
folder_path = 'price'  # Ganti dengan path ke folder Anda

# Dapatkan daftar semua file CSV di folder
csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

# Buat list untuk menyimpan DataFrame dari setiap CSV
dataframes = []

# Baca dan tambahkan setiap CSV ke dalam list DataFrame
for file in csv_files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path)
    dataframes.append(df)

# Gabungkan semua DataFrame menjadi satu
combined_fpl_price_df = pd.concat(dataframes, ignore_index=True)
combined_fpl_price_df.to_csv('combined-fpl-price.csv', index=False)

### Create 'home_team', 'away_team', & 'was_home'

In [6]:
fixtures_2425 = pd.read_csv('../fixtures/fixtures-2024-25.csv')

In [7]:
import pandas as pd

# List untuk menyimpan hasil merge
merged_data = []

# Dapatkan semua GW yang ada
gameweeks = combined_fpl_price_df['GW'].unique()

# Loop melalui setiap GW
for gw in gameweeks:
    # Filter data berdasarkan GW
    gw_fixtures = fixtures_2425[fixtures_2425['GW'] == gw]

    # Merge berdasarkan 'away_team'
    merge_away = pd.merge(combined_fpl_price_df[combined_fpl_price_df['GW'] == gw], gw_fixtures, left_on='team', right_on='away_team')
    merge_away['opponent_team'] = merge_away['home_team']
    merge_away['was_home'] = 0
    merge_away.drop(columns=['away_team', 'home_team', 'GW_y'], inplace=True)

    # Merge berdasarkan 'home_team'
    merge_home = pd.merge(combined_fpl_price_df[combined_fpl_price_df['GW'] == gw], gw_fixtures, left_on='team', right_on='home_team')
    merge_home['opponent_team'] = merge_home['away_team']
    merge_home['was_home'] = 1
    merge_home.drop(columns=['away_team', 'home_team', 'GW_y'], inplace=True)

    # Gabungkan hasil untuk GW ini
    merged_data.append(pd.concat([merge_away, merge_home], ignore_index=True))

# Gabungkan semua GW
fpl_price_df = pd.concat(merged_data, ignore_index=True)
fpl_price_df.rename(columns={'GW_x': 'GW'}, inplace=True)

In [8]:
fpl_price_df

,name,team,price,condition,position,GW,opponent_team,was_home
0,A.Becker,Liverpool,5.5,1.0,GK,1,Ipswich Town,0
1,Ederson M.,Manchester City,5.5,1.0,GK,1,Chelsea,0
2,Steele,Brighton & Hove Albion,4.5,1.0,GK,1,Everton,0
3,Vicario,Tottenham Hotspur,5.0,1.0,GK,1,Leicester City,0
4,Neto,Bournemouth,4.5,1.0,GK,1,Nottingham Forest,0
...,...,...,...,...,...,...,...,...
2277,Onuachu,Southampton,5.0,1.0,FWD,4,Manchester United,1
2278,Stewart,Southampton,4.5,0.5,FWD,4,Manchester United,1
2279,Lankshear,Tottenham Hotspur,4.5,1.0,FWD,4,Arsenal,1
2280,Chiwome,Wolverhampton Wanderers,4.5,0.0,FWD,4,Newcastle United,1


In [9]:
fpl_price_df.to_csv('combined-fpl-price.csv', index=False)